In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
from matplotlib.pylab import rcParams
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
from tensorflow.keras import regularizers
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model

Using TensorFlow backend.


In [2]:
df = pd.read_csv('featurefaultyA.csv')
df['class'] = 1
df_h = pd.read_csv('featureHealthyA.csv')
df_h['class'] = 0

data = pd.concat([df, df_h], axis = 0)
y = data.pop('class')


data_y=np_utils.to_categorical(y)


In [3]:
# 從資料集切分為訓練集以及測試集，以便評估模型
x_train, x_test, y_train, y_test \
    = train_test_split(data,
                       data_y,
                       test_size=0.4,)

print('training set data dimension')
print(x_train.shape)
print(y_train.shape)
print('---------------------------------------')
print('training set: {}'.format(len(x_train)))
print('testing set: {}'.format(len(x_test)))

training set data dimension
(12, 30)
(12, 2)
---------------------------------------
training set: 12
testing set: 8


In [4]:
class CosFace(L.Layer):
    def __init__(self, num_classes=10, scale=30.0, margin=0.35, **kwargs):
        super(CosFace, self).__init__(**kwargs)
        self.num_classes = num_classes
        self.scale = scale
        self.margin = margin

    def build(self, input_shape):
        self.W = self.add_weight(name="W",
                                 shape=(input_shape[0][-1], self.num_classes),
                                 initializer="glorot_uniform",
                                 trainable=True)

    def call(self, inputs):
        # get embeddings and one hot labels from inputs
        embeddings, onehot_labels = inputs
        # normalize final W layer
        W = tf.nn.l2_normalize(self.W, axis=0)
        # get logits from multiplying embeddings (batch_size, embedding_size) and W (embedding_size, num_classes)
        logits = tf.matmul(embeddings, W)
        # subtract margin from logits
        target_logits = logits - self.margin
        # get cross entropy
        logits = logits * (1 - onehot_labels) + target_logits * onehot_labels
        # apply scaling
        logits = logits * self.scale
        # get class probability distribution
        predictions = tf.nn.softmax(logits)
        return predictions

    def compute_output_shape(self, input_shape):
        return (None, self.num_classes)

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            "num_classes": self.num_classes,
            "scale": self.scale,
            "margin": self.margin})
        return config

In [5]:
x_train.shape[1]

30

In [32]:
num_classes = 2
embedding_size = 512
# pretrained_model = load_model(os.path.join(os.path.abspath(
#     ""), "models", "vggface2_train_softmax_" + str(embedding_size) + "d.h5"))
# pretrained_model = model


onehot_labels = L.Input(shape=(num_classes,))
# get feature vector extracted using DenseNet
# feature_extractor = Model(inputs=pretrained_model.input,
#                           outputs=model.get_layer("flatten").output)


inputs = L.Input(shape = (x_train.shape[1],))
x = L.Dense(128, activation='tanh',
#           input_shape=(x_train.shape[1],),
            kernel_initializer="glorot_normal",
            name = 'dense_0'
#           activity_regularizer=regularizers.l2(1e-5)
           )(inputs)
            
# x = L.Flatten()(feature_extractor.output)
# BN-Dropout-FC-BN
x = L.BatchNormalization(name = 'BN_0')(x)
x = L.Dropout(0.25)(x)
x = L.Dense(embedding_size, activation="relu")(x)
x = L.BatchNormalization(name='BN_vector')(x)
# make embeddings unit length
embeddings = L.Lambda(lambda x: tf.nn.l2_normalize(x, axis=1))(x)

predictions = CosFace(num_classes=num_classes)([embeddings, onehot_labels])
# model = Model(inputs=[feature_extractor.input,onehot_labels], 
#                       outputs=predictions)
model = Model(inputs=[inputs,onehot_labels], 
                      outputs=predictions)

model.compile("adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_24 (InputLayer)           [(None, 30)]         0                                            
__________________________________________________________________________________________________
dense_0 (Dense)                 (None, 128)          3968        input_24[0][0]                   
__________________________________________________________________________________________________
BN_0 (BatchNormalization)       (None, 128)          512         dense_0[0][0]                    
__________________________________________________________________________________________________
dropout_6 (Dropout)             (None, 128)          0           BN_0[0][0]                       
___________________________________________________________________________________________

In [33]:
batch_size = 32  # 每次看 batch_size 筆的資料就更新權重
epochs = 15# 一個 epoch 會看過一次所有的資料


# 訓練模型
model_history = model.fit(x=[x_train, y_train], y=y_train, # 進行訓練 學習分布
                          batch_size=batch_size,
                          epochs=epochs,
#                           validation_data=(x_test, y_test),
                          shuffle=True,
                          verbose = False
                         )

In [42]:
feature_extractor = Model(inputs=model.input,
                          outputs=model.get_layer("BN_vector").output)

# inputs = L.Input(shape = (x_train.shape[1],))

cls_layer = L.Dense(num_classes, activation='softmax')(feature_extractor.output)
inf_model = Model(inputs = model.inputs, outputs = cls_layer)
inf_model.compile("adam", loss="categorical_crossentropy", metrics=["accuracy"])

num_classes = 2

for layer in inf_model.layers:
    layer.traninable = False
for layer in inf_model.layers[-1:]:
    layer.trainable = True
model.summary()
batch_size = 32  # 每次看 batch_size 筆的資料就更新權重
epochs = 15 # 一個 epoch 會看過一次所有的資料


# 訓練模型
model_history = inf_model.fit(x=[x_train, None], y=y_train, # 進行訓練 學習分布
                          batch_size=batch_size,
                          epochs=epochs,
                          validation_data=(x_test, y_test),
                          shuffle=True
                         )

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_24 (InputLayer)           [(None, 30)]         0                                            
__________________________________________________________________________________________________
dense_0 (Dense)                 (None, 128)          3968        input_24[0][0]                   
__________________________________________________________________________________________________
BN_0 (BatchNormalization)       (None, 128)          512         dense_0[0][0]                    
__________________________________________________________________________________________________
dropout_6 (Dropout)             (None, 128)          0           BN_0[0][0]                       
___________________________________________________________________________________________

AttributeError: 'NoneType' object has no attribute 'shape'

In [36]:
inputs = L.Input(shape = (x_train.shape[1],))

x = model.get_layer('dense_0')()
x = L.BatchNormalization(name='BN_vector')(x)
x = L.Dropout(0.25)(x)
x = L.Dense(embedding_size, activation="relu")(x)
x = L.BatchNormalization(name='BN_vector')(x)

cls_layer = L.Dense(num_classes, activation='softmax')(x)

inf_model = Model(inputs = inputs, outputs = cls_layer)
inf_model.compile("adam", loss="categorical_crossentropy", metrics=["accuracy"])
# 進行推論
num_classes = 2

for layer in inf_model.layers:
    layer.traninable = False
for layer in inf_model.layers[-1:]:
    layer.trainable = True
model.summary()
batch_size = 32  # 每次看 batch_size 筆的資料就更新權重
epochs = 15 # 一個 epoch 會看過一次所有的資料


# 訓練模型
model_history = inf_model.fit(x=x_train, y=y_train, # 進行訓練 學習分布
                          batch_size=batch_size,
                          epochs=epochs,
                          validation_data=(x_test, y_test),
                          shuffle=True
                         )

ValueError: ('Input has undefined rank:', TensorShape(None))

In [21]:
a = inf_model.predict([x_train, np.array([[0, 0]])])
print(a)
y_pred = []
for i in a:
    y_pred.append(np.argmax(i))


[[0.4825076  0.5174924 ]
 [0.47830367 0.5216963 ]
 [0.4804379  0.51956207]
 [0.47366205 0.526338  ]
 [0.48239055 0.5176094 ]
 [0.48256227 0.5174377 ]
 [0.48030272 0.5196973 ]
 [0.48823005 0.5117699 ]
 [0.47971672 0.5202832 ]
 [0.52339005 0.47660992]
 [0.4825929  0.5174071 ]
 [0.48629394 0.513706  ]]


In [16]:
y_true = []
for i in y_train:
    y_true.append(np.argmax(i))
#     print(np.argmax(i))

In [17]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_true, y_pred)

(array([0.        , 0.09090909]),
 array([0. , 0.5]),
 array([0.        , 0.15384615]),
 array([10,  2]))

In [9]:
from keras.layers import Input, Dense
from keras.models import Model

# 这部分返回一个张量
inputs = Input(shape=(784,))

# 层的实例是可调用的，它以张量为参数，并且返回一个张量
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# 这部分创建了一个包含输入层和三个全连接层的模型
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_5 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_6 (Dense)              (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________
